In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

#! pip install statsmodels

import statsmodels.api as sm
import statsmodels

main_directory = Path.cwd().parent.as_posix()

In [2]:
df = pd.read_csv(f"{main_directory}/databases/zoopla_data.csv")
df

,type,price,station,beds,baths,receptions,post_code,location,avg_sold_price_12months,avg_type
0,flat,110000.0,1.3,2,1.0,1.0,M8,manchester,162788,101576.0
1,flat,115000.0,1.4,2,1.0,1.0,M8,manchester,162788,101576.0
2,flat,170000.0,0.7,2,1.0,1.0,M8,manchester,162788,101576.0
3,semi-detached,180000.0,1.4,3,1.0,1.0,M8,manchester,162788,208116.0
4,semi-detached,190000.0,1.9,3,1.0,0.0,M8,manchester,162788,208116.0
...,...,...,...,...,...,...,...,...,...,...
6134,detached,600000.0,4.1,4,0.0,0.0,CA4,cumbria,272071,334295.0
6135,detached,375000.0,1.7,4,1.0,1.0,CA4,carlisle,272071,334295.0
6136,detached,447500.0,0.4,4,0.0,0.0,CA4,carlisle,272071,334295.0
6137,detached,350000.0,1.4,5,0.0,0.0,CA4,carlisle,272071,334295.0


In [3]:
def feature_eng(df):
    df = pd.get_dummies(df, columns = ['type'], drop_first = True, prefix = 'type')
    #df = pd.get_dummies(df, columns = ['location'], drop_first = True, prefix = 'loc')
    df = sm.add_constant(df)
    return df

In [4]:
df

,type,price,station,beds,baths,receptions,post_code,location,avg_sold_price_12months,avg_type
0,flat,110000.0,1.3,2,1.0,1.0,M8,manchester,162788,101576.0
1,flat,115000.0,1.4,2,1.0,1.0,M8,manchester,162788,101576.0
2,flat,170000.0,0.7,2,1.0,1.0,M8,manchester,162788,101576.0
3,semi-detached,180000.0,1.4,3,1.0,1.0,M8,manchester,162788,208116.0
4,semi-detached,190000.0,1.9,3,1.0,0.0,M8,manchester,162788,208116.0
...,...,...,...,...,...,...,...,...,...,...
6134,detached,600000.0,4.1,4,0.0,0.0,CA4,cumbria,272071,334295.0
6135,detached,375000.0,1.7,4,1.0,1.0,CA4,carlisle,272071,334295.0
6136,detached,447500.0,0.4,4,0.0,0.0,CA4,carlisle,272071,334295.0
6137,detached,350000.0,1.4,5,0.0,0.0,CA4,carlisle,272071,334295.0


In [5]:
df = feature_eng(df)
df['avg_sold_price_12months']=(df['avg_sold_price_12months']-df['avg_sold_price_12months'].mean())/df['avg_sold_price_12months'].std()
df['avg_type']=(df['avg_type']-df['avg_type'].mean())/df['avg_type'].std()

feature_cols = list(df.columns)
feature_cols.remove('price')
feature_cols.remove('post_code') #Remove address -> not needed in the model
feature_cols.remove('station') #Remove address -> not needed in the model
feature_cols.remove('location') #Remove address -> not needed in the model


In [6]:
X = df[feature_cols]
X = sm.add_constant(X)
y = df['price']

lin_reg = sm.OLS(y, X)
results = lin_reg.fit()
df['y_pred'] = results.predict(X)

rmse = statsmodels.tools.eval_measures.rmse(y, df['y_pred'])

print(f'the RMSE is {rmse}')
print(results.params)
results.summary()

the RMSE is 108237.07004441382
const                     -197368.851277
beds                        69863.913055
baths                       27893.668056
receptions                  15521.403777
avg_sold_price_12months      5181.771891
avg_type                        0.803776
type_flat                  103352.077437
type_semi-detached           3311.094007
type_terraced                5637.802496
dtype: float64


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.569
Method:                 Least Squares   F-statistic:                     1015.
Date:                Wed, 12 Jan 2022   Prob (F-statistic):               0.00
Time:                        18:18:32   Log-Likelihood:                -79875.
No. Observations:                6139   AIC:                         1.598e+05
Df Residuals:                    6130   BIC:                         1.598e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                   -1.974e+05   1.39e+04    -14.155      0.000   -2.25e+05    -1.7e+05
beds                     6.986e+04   2158.617     32.365      0.000    6.56e+04    7.41e+04
baths                    2.789e+04   2173.297     12.835      0.000    2.36e+04    3.22e+04
receptions               1.552e+04   1890.404      8.211      0.000    1.18e+04    1.92e+04
avg_sold_price_12months  5181.7719   2081.534      2.489      0.013    1101.234    9262.310
avg_type                    0.8038      0.032     25.100      0.000       0.741       0.867
type_flat                1.034e+05   7984.607     12.944      0.000    8.77e+04    1.19e+05
type_semi-detached       3311.0940   6424.333      0.515      0.606   -9282.853    1.59e+04
type_terraced            5637.8025   7839.073      0.719      0.472   -9729.532     2.1e+04
==============================================================================
Omnibus:                     5661.719   Durbin-Watson:                   1.674
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           496158.468
Skew:                           4.118   Prob(JB):                         0.00
Kurtosis:                      46.265   Cond. No.                     3.00e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  3e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""